In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import os
import vectorbt as vbt

# Filter warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)  
warnings.filterwarnings('ignore', category=FutureWarning)   
warnings.filterwarnings('ignore', category=UserWarning)     
np.seterr(all='ignore')  
pd.options.mode.chained_assignment = None  

# Add parent directory to path to import strategy modules
import sys
sys.path.append('..')

from example_scripts.strategy_0_buy_and_hold import BuyAndHoldStrategy
from example_scripts.strategy_1_momentum import DualMomentumStrategy
from example_scripts.strategy_2_regime import MacroRegimeStrategy
from example_scripts.strategy_3_mean_reversion import MeanReversionStrategy
from example_scripts.strategy_4_multi_factor import MultiFactorStrategy
from example_scripts.strategy_5_volatility_regime import VolatilityRegimeStrategy
from example_scripts.strategy_6_adaptive_trend import AdaptiveTrendStrategy
from example_scripts.strategy_8_combined import CombinedStrategy

In [4]:
# Load data
def load_data() -> pd.DataFrame:
    data_path = Path('..') / 'raw_data' / 'df.csv'
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

df = load_data()

# Initialize strategies
strategies = [
    BuyAndHoldStrategy(df),
    DualMomentumStrategy(df),
    MacroRegimeStrategy(df),
    MeanReversionStrategy(df),
    MultiFactorStrategy(df),
    VolatilityRegimeStrategy(df),
    AdaptiveTrendStrategy(df),
    CombinedStrategy(df)
]

In [23]:
import io
import sys
from contextlib import redirect_stdout
import pandas as pd
from datetime import datetime

# Dictionary to store stats for each strategy
all_stats = {}

for strategy in strategies:
    # Capture and suppress the print output from generate_signals
    with redirect_stdout(io.StringIO()):
        signals = strategy.generate_signals()
    
    price = strategy.df[strategy.target_col]
    
    # Make sure price and signals have the same index
    signals = signals.reindex(price.index)
    
    # Generate entries and exits directly from daily signals
    entries = signals & ~signals.shift(1).fillna(False)
    exits = ~signals & signals.shift(1).fillna(False)
    
    # Get only the portfolio stats
    pf = vbt.Portfolio.from_signals(
        price,
        entries,
        exits,
        freq='1D',
        init_cash=100,
        size=1.0,  # 100%
        size_type='percent',  # Use percentage of portfolio
        accumulate=False
    )
    # Store just the portfolio stats
    stats_series = pf.stats()
    all_stats[strategy.__class__.__name__] = stats_series

# Convert to DataFrame and sort by Total Return
df_stats = pd.DataFrame(all_stats).sort_values(by='Total Return [%]', axis=1, ascending=False)

# Format specific columns
def format_stats(df):
    formatted_df = df.copy()
    
    # Format dates
    formatted_df.loc['Start'] = formatted_df.loc['Start'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
    formatted_df.loc['End'] = formatted_df.loc['End'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
    
    # Format Total Return
    formatted_df.loc['Total Return [%]'] = formatted_df.loc['Total Return [%]'].apply(
        lambda x: f"{x:.2f}%" if pd.notnull(x) else x
    )
    
    # Format Start Value and End Value to 2 decimal points
    for row in ['Start Value', 'End Value']:
        formatted_df.loc[row] = formatted_df.loc[row].apply(lambda x: f"{x:.2f}" if pd.notnull(x) else x)
    
    # Format all duration-related fields to just days
    duration_rows = ['Avg Winning Trade Duration', 'Avg Losing Trade Duration', 'Max Drawdown Duration', 'Period']
    for row in duration_rows:
        if row in formatted_df.index:
            formatted_df.loc[row] = formatted_df.loc[row].apply(
                lambda x: f"{pd.Timedelta(x).days} days" if pd.notnull(x) else x
            )
    
    # Round all other numeric values to 2 decimal points
    numeric_rows = [idx for idx in formatted_df.index 
                   if idx not in ['Start', 'End', 'Total Return [%]'] + duration_rows + ['Start Value', 'End Value']]
    for row in numeric_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{float(x):.2f}" if pd.notnull(x) and not isinstance(x, pd.Timedelta) else x
        )
    
    return formatted_df

# Apply formatting
formatted_df = format_stats(df_stats)
formatted_df

,AdaptiveTrendStrategy,CombinedStrategy,BuyAndHoldStrategy,MultiFactorStrategy,MacroRegimeStrategy,VolatilityRegimeStrategy,MeanReversionStrategy,DualMomentumStrategy
Start,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002
End,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024
Period,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days
Start Value,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
End Value,225.32,147.60,134.69,127.45,125.60,109.89,100.23,99.87
Total Return [%],125.32%,47.60%,34.69%,27.45%,25.60%,9.89%,0.23%,-0.13%
Benchmark Return [%],34.69,34.69,34.69,34.69,34.69,34.69,34.69,34.69
Max Gross Exposure [%],100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
Total Fees Paid,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Max Drawdown [%],0.73,0.44,15.48,3.01,0.84,1.37,0.67,1.63


In [24]:
import io
import sys
from contextlib import redirect_stdout
import pandas as pd
from datetime import datetime

# Dictionary to store stats for each strategy
all_stats = {}

for strategy in strategies:
    # Capture and suppress the print output from generate_signals
    with redirect_stdout(io.StringIO()):
        signals = strategy.generate_signals()
    
    price = strategy.df[strategy.target_col]
    
    # Make sure price and signals have the same index
    signals = signals.reindex(price.index)
    
    # Resample to monthly, then forward fill back to daily
    monthly_signals = signals.resample('M').last()
    resampled_signals = monthly_signals.reindex(price.index, method='ffill')
    
    # Generate entries and exits
    entries = resampled_signals & ~resampled_signals.shift(1).fillna(False)
    exits = ~resampled_signals & resampled_signals.shift(1).fillna(False)
    
    # Get only the portfolio stats
    pf = vbt.Portfolio.from_signals(
        price,
        entries,
        exits,
        freq='1D',
        init_cash=100,
        size=1.0,
        size_type='percent',
        accumulate=False
    )
    # Store just the portfolio stats
    stats_series = pf.stats()
    all_stats[strategy.__class__.__name__] = stats_series

# Convert to DataFrame and sort by Total Return
df_stats = pd.DataFrame(all_stats).sort_values(by='Total Return [%]', axis=1, ascending=False)

# Format specific columns
def format_stats(df):
    formatted_df = df.copy()
    
    # Format dates
    formatted_df.loc['Start'] = formatted_df.loc['Start'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
    formatted_df.loc['End'] = formatted_df.loc['End'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
    
    # Format Total Return
    formatted_df.loc['Total Return [%]'] = formatted_df.loc['Total Return [%]'].apply(
        lambda x: f"{x:.2f}%" if pd.notnull(x) else x
    )
    
    # Format Start Value and End Value to 2 decimal points
    for row in ['Start Value', 'End Value']:
        formatted_df.loc[row] = formatted_df.loc[row].apply(lambda x: f"{x:.2f}" if pd.notnull(x) else x)
    
    # Format all duration-related fields to just days
    duration_rows = ['Avg Winning Trade Duration', 'Avg Losing Trade Duration', 'Max Drawdown Duration', 'Period']
    for row in duration_rows:
        if row in formatted_df.index:
            formatted_df.loc[row] = formatted_df.loc[row].apply(
                lambda x: f"{pd.Timedelta(x).days} days" if pd.notnull(x) else x
            )
    
    # Round all other numeric values to 2 decimal points
    numeric_rows = [idx for idx in formatted_df.index 
                   if idx not in ['Start', 'End', 'Total Return [%]'] + duration_rows + ['Start Value', 'End Value']]
    for row in numeric_rows:
        formatted_df.loc[row] = formatted_df.loc[row].apply(
            lambda x: f"{float(x):.2f}" if pd.notnull(x) and not isinstance(x, pd.Timedelta) else x
        )
    
    return formatted_df

# Apply formatting
formatted_df = format_stats(df_stats)
formatted_df

,AdaptiveTrendStrategy,BuyAndHoldStrategy,MacroRegimeStrategy,CombinedStrategy,MultiFactorStrategy,VolatilityRegimeStrategy,MeanReversionStrategy,DualMomentumStrategy
Start,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002,10/31/2002
End,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024,12/27/2024
Period,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days,5562 days
Start Value,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
End Value,147.56,134.69,119.22,115.86,113.23,100.92,98.36,97.47
Total Return [%],47.56%,34.69%,19.22%,15.86%,13.23%,0.92%,-1.64%,-2.53%
Benchmark Return [%],34.69,34.69,34.69,34.69,34.69,34.69,34.69,34.69
Max Gross Exposure [%],100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
Total Fees Paid,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Max Drawdown [%],9.77,15.48,1.26,0.70,3.70,2.46,2.03,3.28


In [26]:
import io
import sys
from contextlib import redirect_stdout
import pandas as pd
from datetime import datetime
import vectorbt as vbt

# Dictionary to store annualized returns for each strategy
annualized_returns = {}

for strategy in strategies:
    # Capture and suppress the print output from generate_signals
    with redirect_stdout(io.StringIO()):
        signals = strategy.generate_signals()
    
    price = strategy.df[strategy.target_col]
    
    # Make sure price and signals have the same index
    signals = signals.reindex(price.index)
    
    # Generate entries and exits directly from daily signals
    entries = signals & ~signals.shift(1).fillna(False)
    exits = ~signals & signals.shift(1).fillna(False)
    
    # Get portfolio with explicit frequency
    pf = vbt.Portfolio.from_signals(
        price,
        entries,
        exits,
        freq='1D',
        init_cash=100,
        size=1.0,
        size_type='percent',
        accumulate=False
    )
    
    # Calculate annualized returns with explicit year frequency
    returns = pf.returns()
    ann_return = returns.vbt.returns(freq='1D', year_freq='365D').annualized()
    annualized_returns[strategy.__class__.__name__] = ann_return *100 # Convert to percentage for display

# Convert to DataFrame and sort by annualized return
df_returns = pd.DataFrame.from_dict(annualized_returns, orient='index', columns=['Annualized Return'])
df_returns = df_returns.sort_values('Annualized Return', ascending=False)

# Format the percentage values and center-align
df_returns['Annualized Return'] = df_returns['Annualized Return'].apply(lambda x: f"{x:.2f}%")
df_returns = df_returns.style.set_properties(**{
    'text-align': 'center'
}).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

# Display the results
df_returns

,Annualized Return
AdaptiveTrendStrategy,5.48%
CombinedStrategy,2.59%
BuyAndHoldStrategy,1.97%
MultiFactorStrategy,1.60%
MacroRegimeStrategy,1.51%
VolatilityRegimeStrategy,0.62%
MeanReversionStrategy,0.01%
DualMomentumStrategy,-0.01%
